In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import tensorflow as tf
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from tensorflow import *
from tensorflow.keras import *
from sklearn.model_selection import *

np.random.seed(111)
tf.random.set_seed(111)



In [4]:
#Wczytywanie danych zestaw Bakterie
df = pd.read_csv("https://raw.githubusercontent.com/paq88/Fish_neural_network_classifier/refs/heads/main/gut_based/Data/gut_full_feature_table.csv")
df.head()

HTTPError: HTTP Error 404: Not Found

Wyjaśnienie numerów stawów

| number | setup | pond numbers | water supplementation | feed suplementation |
|---|---|---|---|---|
|0|control | 8,14,18,24,29|NO|NO|
|1|set 1 | 12,16,19,23,26|Em farma|NO|
|2|set 2 | 10,21,28,30,32|Em farma|EM|
|3|set 3 | 9,13,17,20,27|EM|NO|
|4|set 4 | 11,15,22,25,31|EM|EM|

In [ ]:
#Sprawdzanie typów danych
df.dtypes.value_counts()

In [ ]:
# Brakujące obserwacje
df.isna().sum().plot()


In [ ]:
df[df.isnull()] = 0
#df.fillna(0,inplace = True)
#df.replace(np.nan,0)



df.isna().sum().plot()


In [ ]:
# wartosci min-max dla kolumn
df.iloc[:,1:].min().plot()


In [ ]:
df.iloc[:,1:].max().plot()

In [8]:
# Normalizacja
predyktory = df.columns[1:]
df[predyktory] = df[predyktory]/df[predyktory].max()
df[predyktory] = normalize(df[predyktory],norm = 'max', axis = 0)

In [ ]:
codes = {0 : [8,14,18,24,29], 1: [12,16,19,23,26], 2 : [10,21,28,30,32], 3 : [9,13,17,20,27], 4 : [11,15,22,25,31]}

ponds = df['pond'].str.replace('Gut_S','')

klasy = []
for x in ponds:
    x = int(x)
    for key in codes:
        if x in codes[key]:
            klasy.append(key)


df.insert(1,'klasy',klasy)


In [10]:
train, test = train_test_split(df, train_size = 0.8,stratify = df['klasy'])

In [ ]:
train.shape, test.shape

In [ ]:
train.shape, test.shape
test['klasy'].value_counts()

In [13]:
X_train =  train[predyktory].values
y_train = train['klasy'].values
X_test =  test[predyktory].values
y_test = test['klasy'].values

In [ ]:
X_train.shape,X_test.shape

In [ ]:
y_test = keras.utils.to_categorical(y_test,5)
y_train = keras.utils.to_categorical(y_train,5)
X_test.shape,y_test.shape,X_train.shape,y_train.shape

In [16]:
import xgboost as xgb

In [17]:
dtrain = xgb.DMatrix(X_train, label = train['klasy'])
dtest = xgb.DMatrix(X_test, label = test['klasy'])

parametry = {
    'objective': 'multi:softmax',
    'num_class': 5,
    'max_depth': 3,
    'eta': 0.1,
    'seed': 111
}

x = xgb.train(parametry, dtrain, num_boost_round = 200 )


In [ ]:
predykcje = x.predict(dtest)
predykcje

In [ ]:
true = test['klasy'].values
true

In [ ]:
accuracy_score(predykcje,true)

In [ ]:
cm = confusion_matrix(true, predykcje)
ConfusionMatrixDisplay(cm).plot()

In [ ]:
model1 = keras.Sequential([
    keras.layers.Dense(50, input_shape=(len(predyktory),), activation="relu"),
    keras.layers.Dense(40, activation="relu"),
    keras.layers.Dense(30, activation="sigmoid"),
    keras.layers.Dense(20, activation="sigmoid"),
    keras.layers.Dense(5, activation="softmax")
])

model1.summary()

In [23]:
loss = "categorical_crossentropy"

metryka = ['accuracy']
optimizer = keras.optimizers.Adam(learning_rate = 0.01)
model1.compile(loss = loss, metrics = metryka, optimizer = optimizer)

In [24]:
ES = keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 10
)

In [ ]:
history1 = model1.fit(X_train, y_train, epochs = 200, shuffle = True, validation_data = [X_test, y_test],callbacks = ES)

In [ ]:
predykcja_fnn = model1.predict(X_test)

In [ ]:
predykcje_class = []
for x in predykcja_fnn:
  x = list(x)

  clas = x.index(max(x))
  predykcje_class.append(clas)

predykcje_class

In [ ]:
predykcja_fnn.argmax(axis = 1)

In [29]:
true_class = y_test.argmax(axis = 1)

In [ ]:
accuracy_score(true_class,predykcje_class)

In [ ]:
pd.DataFrame({'val_loss':history1.history['val_loss'], 'loss' : history1.history['loss']}).plot()


In [ ]:
pd.DataFrame({'val_acc':history1.history['val_accuracy'], 'acc' : history1.history['accuracy']}).plot()


In [ ]:
cm = confusion_matrix(true_class, predykcje_class)
ConfusionMatrixDisplay(cm).plot()

In [ ]:
from tpot import TPOTClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load the dataset
data = df
X = df[predyktory]
y = df['klasy']

tpot_config = {
    'xgboost.XGBClassifier': {
        'n_estimators': [10, 50],
        'max_depth': [2, 3, 4],
        'learning_rate': [0.01, 0.1, 0.2],
        'tree_method': ['gpu_hist'],  
        'booster': ['gbtree', 'dart'],
    }
}

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Initialize and train the TPOT model
tpot = TPOTClassifier(verbosity=2, generations=5, population_size=20,n_jobs=-1,scoring='accuracy',config_dict=tpot_config)
tpot.fit(X_train, y_train)

# Evaluate the model
accuracy = tpot.score(X_test, y_test)
print(f"Test Accuracy: {accuracy}")
